<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#概要" data-toc-modified-id="概要-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>概要</a></span></li><li><span><a href="#2018-12-18" data-toc-modified-id="2018-12-18-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2018-12-18</a></span><ul class="toc-item"><li><span><a href="#検索組織の機械学習実行基盤-|-リクルートテクノロジーズ　メンバーズブログ" data-toc-modified-id="検索組織の機械学習実行基盤-|-リクルートテクノロジーズ　メンバーズブログ-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>検索組織の機械学習実行基盤 | リクルートテクノロジーズ　メンバーズブログ</a></span><ul class="toc-item"><li><span><a href="#登場キーワードについての調査" data-toc-modified-id="登場キーワードについての調査-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>登場キーワードについての調査</a></span><ul class="toc-item"><li><span><a href="#DRONE-:-CI/CD-のパイプライン処理" data-toc-modified-id="DRONE-:-CI/CD-のパイプライン処理-2.1.1.1"><span class="toc-item-num">2.1.1.1&nbsp;&nbsp;</span>DRONE : CI/CD のパイプライン処理</a></span></li><li><span><a href="#DigDag-:-機械学習処理システム向けのパイプライン処理ツール" data-toc-modified-id="DigDag-:-機械学習処理システム向けのパイプライン処理ツール-2.1.1.2"><span class="toc-item-num">2.1.1.2&nbsp;&nbsp;</span>DigDag : 機械学習処理システム向けのパイプライン処理ツール</a></span></li><li><span><a href="#.dig-ファイルのサンプル集" data-toc-modified-id=".dig-ファイルのサンプル集-2.1.1.3"><span class="toc-item-num">2.1.1.3&nbsp;&nbsp;</span>.dig ファイルのサンプル集</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#2018-12-19" data-toc-modified-id="2018-12-19-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2018-12-19</a></span><ul class="toc-item"><li><span><a href="#機械学習プロジェクトが失敗する9つの理由---六本木で働くデータサイエンティストのブログ" data-toc-modified-id="機械学習プロジェクトが失敗する9つの理由---六本木で働くデータサイエンティストのブログ-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>機械学習プロジェクトが失敗する9つの理由 - 六本木で働くデータサイエンティストのブログ</a></span><ul class="toc-item"><li><span><a href="#ML-Ops-のポイント" data-toc-modified-id="ML-Ops-のポイント-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>ML Ops のポイント</a></span></li><li><span><a href="#機械学習プロジェクトが失敗する理由" data-toc-modified-id="機械学習プロジェクトが失敗する理由-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>機械学習プロジェクトが失敗する理由</a></span><ul class="toc-item"><li><span><a href="#課題設定の誤り" data-toc-modified-id="課題設定の誤り-3.1.2.1"><span class="toc-item-num">3.1.2.1&nbsp;&nbsp;</span>課題設定の誤り</a></span></li><li><span><a href="#間違った問題に機械学習を使っている" data-toc-modified-id="間違った問題に機械学習を使っている-3.1.2.2"><span class="toc-item-num">3.1.2.2&nbsp;&nbsp;</span>間違った問題に機械学習を使っている</a></span></li><li><span><a href="#十分なデータがない" data-toc-modified-id="十分なデータがない-3.1.2.3"><span class="toc-item-num">3.1.2.3&nbsp;&nbsp;</span>十分なデータがない</a></span></li><li><span><a href="#適切なデータがない（データのクオリティが低い）" data-toc-modified-id="適切なデータがない（データのクオリティが低い）-3.1.2.4"><span class="toc-item-num">3.1.2.4&nbsp;&nbsp;</span>適切なデータがない（データのクオリティが低い）</a></span></li><li><span><a href="#(不要な）データが多すぎる" data-toc-modified-id="(不要な）データが多すぎる-3.1.2.5"><span class="toc-item-num">3.1.2.5&nbsp;&nbsp;</span>(不要な）データが多すぎる</a></span></li><li><span><a href="#人材の配置が適材適所ではない" data-toc-modified-id="人材の配置が適材適所ではない-3.1.2.6"><span class="toc-item-num">3.1.2.6&nbsp;&nbsp;</span>人材の配置が適材適所ではない</a></span></li><li><span><a href="#間違ったツールを使っている" data-toc-modified-id="間違ったツールを使っている-3.1.2.7"><span class="toc-item-num">3.1.2.7&nbsp;&nbsp;</span>間違ったツールを使っている</a></span></li><li><span><a href="#適切なモデルを使っていない" data-toc-modified-id="適切なモデルを使っていない-3.1.2.8"><span class="toc-item-num">3.1.2.8&nbsp;&nbsp;</span>適切なモデルを使っていない</a></span></li><li><span><a href="#適切な評価指標を使っていない" data-toc-modified-id="適切な評価指標を使っていない-3.1.2.9"><span class="toc-item-num">3.1.2.9&nbsp;&nbsp;</span>適切な評価指標を使っていない</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#2018-12-25" data-toc-modified-id="2018-12-25-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>2018-12-25</a></span><ul class="toc-item"><li><span><a href="#データサイエンティスト・機械学習エンジニアは常に最前線にいるべき理由" data-toc-modified-id="データサイエンティスト・機械学習エンジニアは常に最前線にいるべき理由-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>データサイエンティスト・機械学習エンジニアは常に最前線にいるべき理由</a></span><ul class="toc-item"><li><span><a href="#自分のキャリアと会社を切り離して考えるべき時代" data-toc-modified-id="自分のキャリアと会社を切り離して考えるべき時代-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>自分のキャリアと会社を切り離して考えるべき時代</a></span></li><li><span><a href="#データサイエンティスト・機械学習エンジニアというキャリアの信用と独自性を養う" data-toc-modified-id="データサイエンティスト・機械学習エンジニアというキャリアの信用と独自性を養う-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>データサイエンティスト・機械学習エンジニアというキャリアの信用と独自性を養う</a></span><ul class="toc-item"><li><span><a href="#データサイエンティスト・機械学習エンジニア像（ざっくり＠２０１８）" data-toc-modified-id="データサイエンティスト・機械学習エンジニア像（ざっくり＠２０１８）-4.1.2.1"><span class="toc-item-num">4.1.2.1&nbsp;&nbsp;</span>データサイエンティスト・機械学習エンジニア像（ざっくり＠２０１８）</a></span></li><li><span><a href="#なんちゃってデータサイエンティスト" data-toc-modified-id="なんちゃってデータサイエンティスト-4.1.2.2"><span class="toc-item-num">4.1.2.2&nbsp;&nbsp;</span>なんちゃってデータサイエンティスト</a></span></li><li><span><a href="#Excelに固執せず、スケールするテクノロジーを" data-toc-modified-id="Excelに固執せず、スケールするテクノロジーを-4.1.2.3"><span class="toc-item-num">4.1.2.3&nbsp;&nbsp;</span>Excelに固執せず、スケールするテクノロジーを</a></span></li></ul></li><li><span><a href="#データサイエンス・機械学習導入の意義" data-toc-modified-id="データサイエンス・機械学習導入の意義-4.1.3"><span class="toc-item-num">4.1.3&nbsp;&nbsp;</span>データサイエンス・機械学習導入の意義</a></span></li><li><span><a href="#時代の流れに反しない仕事のやり方を" data-toc-modified-id="時代の流れに反しない仕事のやり方を-4.1.4"><span class="toc-item-num">4.1.4&nbsp;&nbsp;</span>時代の流れに反しない仕事のやり方を</a></span></li><li><span><a href="#メモ" data-toc-modified-id="メモ-4.1.5"><span class="toc-item-num">4.1.5&nbsp;&nbsp;</span>メモ</a></span></li></ul></li></ul></li></ul></div>

# 概要
機械学習について調べた履歴を記録する。  
Jupyter Notebook の便利な機能は、履歴作成に当たって大変助かる。(Table of Contents の自動生成など)
まずは、細かい整理はせず、片っ端から記録するというスタイルで始めるとしよう。  


# 2018-12-18

## 検索組織の機械学習実行基盤 | リクルートテクノロジーズ　メンバーズブログ
  https://recruit-tech.co.jp/blog/2018/09/10/qass_ml/
  
* 機械学習システムのアーキテクチャ図があるので参考になる。
    * ![アーキテクチャ](https://s3-ap-northeast-1.amazonaws.com/prod-rtc-blog/wp-content/uploads/2018/08/31193348/flya.png)
* ソフトウェア資産の再利用性を高めるため、Dockerを活用している。
    * Google Container Registry (GCR)に、コンテナを登録している。
* Google Kubernetes Engine (GKE)の上に、"DRONE", "DigDag" といったアプリケーションを構築。
* 機械学習処理の最終出力は、Google Cloud Storage(GCS)上に保存。
* 処理フローの共通化のポイント
    1. モデル部分は共通化しているが、
    2. 前処理部分は、サービスごとに開発する
    
### 登場キーワードについての調査

#### DRONE : CI/CD のパイプライン処理

JenkinsやCircle CIと同じカテゴリのCI/CDツールだが、**Gitイベント** をトリガーにして、**Dockerコンテナ**内部でジョブを実行することに特化したツールである。  
Dockerを利用したマイクロサービス化が普及するにつれて、DRONEの普及も進んでいる模様。  

1. 対象Gitリポジトリのルートディレクトリにある **.drone.yml** を読み込んで、パイプライン処理を実行する。
   .drone.ymlの例
   ***
```
pipeline:
  # ステップは上から順に実行されます（順序付きmap）。
  hello:
    # ステップ毎にイメージを指定
    image: alpine:3.6
    # Dockerコンテナ起動時のエントリポイントとしてコマンドが実行されます。
    # 各コマンドは同じシェルで実行されます。
    commands:
      - echo 　"Hello World"
```
   ***

参照: https://engineering.linecorp.com/ja/blog/go-oss-ci-tool-drone-replaces-jenkins/


#### DigDag : 機械学習処理システム向けのパイプライン処理ツール

機械学習処理システムは、処理フローが複雑化しやすく（特に前処理）、技術的負債の利子が危険レベルになる傾向がある。  
処理フローを記述しやすく、把握しやすく管理できるのがDigDagとのこと。  
以下のように、処理フローを階層的に記述しやすい。

![DigDag処理フローイメージ](http://docs.digdag.io/_images/grouping-tasks.png)


yml的な記述をする .dig ファイルに処理フローの定義を記述する。

.dig の例
***
```text
# daily_etl.dig
schedule:
  daily>: 00:05:00
 
+foo:
  _retry: 3
 
  +get_foo_metrics1:
    sh>: SESSION_TIME=${session_time} python elasticsearch_client.py foo metrics1 1day > foo_metrics1_daily.csv
  +vaidate_foo_metrics1:
    sh>: bash data_validator.sh foo_metrics1_daily.csv
  +commit_foo_metrics1:
    sh>: bash commit.sh foo_metrics1_daily.csv
  +get_foo_metrics2:
    sh>: SESSION_TIME=${session_time} python elasticsearch_client.py foo metrics2 1day > foo_metrics2_daily.csv
  +vaidate_foo_metrics2:
    sh>: bash data_validator.sh foo_metrics2_daily.csv
  +commit_foo_metrics2:
    sh>: bash commit.sh foo_metrics2_daily.csv

```
***

学習処理を定義する .dig ファイルの例を示す。
***
**daily_train.dig**
```
# daily_train.dig
schedule:
  daily>: 00:10:00
 
+wait_daily_etl:
  require>: daily_etl
 
+foo:
  +train:
    _parallel: true
 
    +train_foo_metrics1:
      sh>: python train.py foo metrics1 foo_metrics1_daily.csv
    +train_foo_metrics2:
      sh>: python train.py foo metrics2 foo_metrics2_daily.csv
 
  +cross_validation:
    _pararell: true
 
    +cross_validation_foo_metrics1:
      sh>: python cross_validation.py foo metrics1
    +cross_validation_foor_metrics2:
      sh>: python cross_validation.py foo metrics2
 
_error:
  sh>: bash notify.sh
```
***

cross_validation.py は、過学習を起こしていたと判断すると、終了コード 1 を返すプログラムとなっている。  
過学習が起きると \_error が呼び出され、通知処理(notify.sh)が起動するようになっている。  


週次の処理をする場合は、以下のような依存関係を前提とした .digファイルを記述する。  
以下の.digファイルは、 requireで指定された daily_train.dig が7回実行されるのを待ってから、foo 以下の処理が実行される。

***
```
# fine_tuning.dig
schedule:
  weekly>: Mon,01:05:00
 
+wait_daily_train:
  loop>: 7
  _do:
    require>: daily_train
    session_time: ${moment(last_session_time).add(i, 'day').format()}
 
+foo:
  +train:
    _parallel: true
 
    +fine_tuning_foo_metrics1:
      sh>: python fine_tuning.py foo metrics1
    +fine_tuning_foo_metrics2:
      sh>: python fine_tuning.py foo metrics2
```
***


定期的にポーリングして、モデルの性能悪化を検出したら、すかさず通知したいときには、以下のような .digファイルを作ると良いという。  
以下では5分ごとに predict.py を実行し、モデルの性能が悪化して exit_code = 1 となったら、_error がキックされ、通知処理(notify.sh)が実行される
。  

*** 
```
# predict.dig
schedule:
  minutes_interval>: 5
 
+foo:
  +predict:
    _parallel: true
 
    +predict_foo_metrics1:
      sh>: python predict.py foo metrics1
    +predict_foo_metrics2:
      sh>: python predict.py foo metrics2
 
_error:
  sh>: bash notify.sh
```
***

#### .dig ファイルのサンプル集

トレジャーデータの以下のサイトで紹介している。  
https://github.com/treasure-data/digdag/tree/master/examples

##### 参照リンク
* [DigDag公式サイト](http://docs.digdag.io/index.html)
* https://www.lifull.blog/entry/2017/09/20/151600

# 2018-12-19

## 機械学習プロジェクトが失敗する9つの理由 - 六本木で働くデータサイエンティストのブログ
出展  https://tjo.hatenablog.com/entry/2018/08/03/080000

### ML Ops のポイント

DevOpsならぬ **"ML Ops"** が普及している。ML Ops で関心があるとされているのは以下の４点。  
1. システムやサービスのどの場所で機械学習を使うのか
2. 機械学習システムの運用方法のポイント
    * 機械学習処理を含むコードのテスト方法
    * 構築したモデルのデプロイ方法
    * モデルとデータの監視
        * 性能が悪化したことを検知して、モデル更新のタイミングを知らせる方法などか？
    * 開発～デプロイプロセスの自動化の方法
        * 先に触れたリクルートテクノロジーズのエントリーが該当するかもしれない
3. 機械学習処理システムの高速化(ロジックのチューニングやGPUの利用など)
4. 機械学習処理システムの開発体制,運用体制

### 機械学習プロジェクトが失敗する理由

#### 課題設定の誤り
不正検知の場合、「過去に人が見つけてきた事例」をベースに不正を定義してしまうと、**「新しい手口の不正」**に対処できなくなってしまう。  
この場合は、**「何が正常なのか」**を学習させるのが適切である。  
異常時のデータが入手困難であれば、大量の正常データから、正常の定義をしっかりと構築し、そこから逸脱する異常値を検知するというわけである。  
> 例えば「正常値データで学習させたAutoencoderで入力データを復元できたら正常値、復元できなかったら異常値」というやり方などは好例かと。

#### 間違った問題に機械学習を使っている
費用対効果を考慮せずに、闇雲に機械学習システムを作ると大変なことになる。  
明確なニーズがあること(=機械学習を用いて解決するだけの**ビジネス的価値が高い問題**であること)を確かめずに、高いコストを払って機械学習システムを構築すると大赤字である。   
> 「2000万円の人件費を浮かせるために1億円かけて機械学習システムを作ったのに、売上高はたったの3000万円」とかだと目も当てられません。

#### 十分なデータがない
そもそも**「データが入手困難/不可能な分野」**に、機械学習システムを投入することが戦略的に誤りである。  
どんなに優秀な重火器（優秀なモデル）をそろえても、弾薬（データ）が無ければ戦うことはできない。


#### 適切なデータがない（データのクオリティが低い）
まさに**"Garbage in, garbage out"**。データの量はあるが、役に立たないか、誤ったデータばかりだと、モデルがいくら優秀でも誤った判断に収束してしまう。  
データ有効性チェックプロセス(Data Validation Process)のための専任部署を立ち上げる企業もあるという。  
ビジネス的価値の高い問題に取り組むのであれば、そうした専門部署を立ち上げても十分割に合うのではないか。

#### (不要な）データが多すぎる
データの量はあっても、手がかりとならないデータばかりだと苦労する。  
「特徴量削減」の苦労である。  
PCAやt-SNEで次元を圧縮するにせよ、特徴量を削減するにせよ、どの特徴量を残すのかは人の手にゆだねられる。  
データを蓄積する段階から、機械学習にかけることを考慮した設計にしておくことが望ましい。  

#### 人材の配置が適材適所ではない
チームが小さい、部門の立ち上げ時期には、データ収集・加工・前処理・仮説検証・モデル構築・プロダクト実装・デプロイまで出来るオールマイティな古巣タックな機械学習開発者/データサイエンティストが数名いればよい。 

だが、チームが大きくなってきたら、ETL特化型、モデル構築特化型など、機械学習分野の中でも特化した人材がいると良い。  
データ基盤を世話するデータアーキテクトは、部門や事業の規模が大きくなってきたら必須になるだろう。  

#### 間違ったツールを使っている
レガシーな既存システムに縛られ、規模がスケールしにくい、従来の技術やシステムに固執するのは危険である。  
将来、高い確率で**技術的負債**になる。  
なるべく新しく、**スケールする技術**を取り入れるべきである。  
* オンプレのデータベースよりは、クラウドのデータウェアハウス(GCPのBigQueryとか)

#### 適切なモデルを使っていない
闇雲にディープラーニングを導入するのは、問題である。適切なチューニングが出来ないと目も当てられなくなる。    
例えば、線形分離可能な問題には線形モデルを適用すべきである。非線形モデルを入れるとおかしくなる。  
機械学習モデルも適材適所である。

#### 適切な評価指標を使っていない
予測する対象、課題に応じて、重視する評価指標を使い分ける必要がある。  
異常検知の場合、大多数が正常なので、精度(accuracy)で言えば常に１００％になってしまう。
実際には、見逃すとリスクが大きいので、対象精度を犠牲にしてでも高い再現率(recall)を達成すべきケースが多いだろう。  




# 2018-12-25

## データサイエンティスト・機械学習エンジニアは常に最前線にいるべき理由

お題となるエントリーはこちらである。  
***
[データサイエンティストや機械学習エンジニアが、可能な限り統計学や機械学習やプログラミングを使って課題を解決するべき3つの理由 - 六本木で働くデータサイエンティストのブログ](https://tjo.hatenablog.com/entry/2018/12/04/190000)
***

### 自分のキャリアと会社を切り離して考えるべき時代
> 私の意見では、個人のキャリアは会社とは切り離して考えるべきであり、従業員は経営者視点（会社が儲かるなら何でもやる）になってはいけない。会社や顧客のために個人のキャリアや生き方（データ分析以外の仕事も何でもやるデータ分析者）を犠牲にするような労働はすべきではない。

データ分析の現場から離れて、パワポ作りや営業的タスクに傾いてゆくと「**おしゃべり課題解決コンサルおじさん**」になってしまうという主張は、尤もである。  
最新の技術や理論にキャッチアップできなくなると、コンサルタントの一人として埋もれてしまうのは容易に想像できる。  

また一つの会社に人生をささげる時代ではなくなり、ポータブルスキル・アビリティーというべき、どこに行っても認識されるし、通用するだけの独立した能力や技能・知見を養っていくべき時代が訪れつつあるような気がしている。  


### データサイエンティスト・機械学習エンジニアというキャリアの信用と独自性を養う

#### データサイエンティスト・機械学習エンジニア像（ざっくり＠２０１８）
> これらの職業はそれぞれ「統計分析」and/or「機械学習」そしてプログラミングで高度に自動化されたプロセスに基づくデータ分析を担う、独自性のある専門職として認識されるようになっていった、というのが僕個人の見解です。

本来的には、統計分析・機械学習に通じており、Python等のプログラミング言語を用いて、データ解析・機械学習システムを構築する能力・経験・知見を持つのが、これらデータサイエンティスト・機械学習エンジニアの姿というべきだろう。  
機械学習エンジニアは、仮説・モデル構築というよりは、構築されたモデル（検証済み）を、実稼動するプロダクトとして落とし込むという、ソフトウェアエンジニアリングよりの役割が、次第に鮮明に求められるようになっている模様。  
分業化が始まっているようだ。（特に、データ分析部門、AI部門の歴史が長い大手IT企業）

#### なんちゃってデータサイエンティスト
> 必然的に統計学も機械学習も況してやプログラミングも出来ない「なんちゃってデータサイエンティスト」が続出し、彼らがレベルの低いアウトプットを繰り返して社会からの幻滅を誘ったことで、第一次ブームは萎んでいったのでした。

なんちゃってデータサイエンティスト、「でーたさいえんてぃすと」というキーワードが、具体的にどういった人物像を示しているのか今まで判然としなかったが、これで具体的になった。  
ビジネス畑出身で、統計学・機械学習・プログラミングのいずれのバックグランドを持たず、分析業務に手を出して、いたずらに周囲を混乱させた人物ということだろう。  
確かに、そうした人物がデータサイエンティストであったら、どうしようもない。  


#### Excelに固執せず、スケールするテクノロジーを
> 「統計学や機械学習やプログラミングを使わずに問題を解決してしまうと、次からそれらを使える機会が失われやすい」
> 前述したように、データサイエンティストや機械学習エンジニアの独自性は「統計学や機械学習やプログラミングを用いて問題を解決できる」点にあります。にもかかわらず、それらを使える機会を失うということはそれらの職業としての独自性を失うということです。

Pythonではなく、Excelベースでデータ分析をさせる会社は危険である。  
扱うデータの規模が拡大した際に、手の打ちようがない。スケール困難である。  
よって、Python、Hadoop+Hive、BigQuery、AWS などスケールが見込めるテクノロジーを用いて、分析基盤や分析プロセスを確立しておくべきであろう。  
Excelに固執する会社は、テクノロジーがもたらすデジタルトランスフォーメーションの波への適応を拒否しているも同じであり、淘汰の運命を受け入れたのに等しいだろう。  

こうしたテクノロジーに立脚した仕事の進め方に軸足を置くことは、データサイエンティストや機械学習エンジニアというキャリアの信用を養い、かつ自分自身を守る上でも、実に合理的であろう。（さらに、データ解析を導入する企業にとっても長期的展望にたって有益である）


### データサイエンス・機械学習導入の意義

* 人の直感的認識を越えた複雑な認識ができるようになる。

> 「属人性」を排し、「再現性」高いデータ分析を行い、「自動化」によって、「スケーラビリティ」の向上を目指す

担当者がいなくなっても大丈夫、安定して再現性が高い分析ができるし、自動化されているので取り扱うデータ量が増えても対応できる（＝スケーラビリティ）


### 時代の流れに反しない仕事のやり方を

> 統計学や機械学習やプログラミングによって課題解決していくことが「好き」な人が、ビジネス上の要請だからという理由でそれらをあえて封印するような仕事のやり方をするべきではない、

データに基づいた意思決定プロセスが、世の中に浸透・普及しつつあるこの時代に、前時代的なアプローチに戻ることは危険な逆走行為であろう。  
古いやり方に固執するのは、基本的には避けるべきである。  
ただし、競争力の源泉となるような本質的な要素であり、AIやデータ分析ではまだ代替できないほど高度なものであれば、尊重すべし。  
対人コミュニケーションや高度な意思決定、創造的プロセスなど、AIではまだ到達できていない高度なものは残るだろう。

それ以外、手順は明確、自動化の可能性が十分あるのに、変化を拒否したいがためだけに、時代の流れに逆行するスタンスを取るのは完全にＮＧだろう。  
無自覚な自殺行為であり、それを指摘し、変えなければ生存が危うくなるだろう。  


### メモ

* 「多変量モデル」＝特徴量を使って、目的変数を予測する、つまり機械学習モデルは多変量モデルに含まれる

> ということで第二次データサイエンティストブームも間も無く崩壊するのだろうなと思っています。同時に人工知能ブーム、そして機械学習エンジニアブームも崩壊するのか否かを、今後は注意深く見守っていく必要がありそうです。。。

このあたりはポジショントークである。分かりやすい。  
データサイエンティスト、機械学習エンジニアの希少性を維持し、陳腐化を防ぐため、やんわりと超間接的にアプローチをしているといったところか。  
